# Búsqueda de parejas letales sintéticas basada en la conservación 
```
Título original: Conservation-based Synthetic Lethal Pair Search
Autor: Taek-Kyun Kim
Creado: 02-07-2022
Cuaderno de Jupyter traducido y modificado por Martín Sende Pombo.
Fecha de modificación: 17-02-2024
Propósito: Obtención de parejas letales sintéticas de los genes de la lista dada, mediante el cribado de levaduras y la información de homología entre humanos y levaduras.
Notas: Según el autor puede ejecutarse en MyBinder.
```
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/isb-cgc/Community-Notebooks/HEAD?labpath=SL-Cloud%2FMyBinder_Authentication.ipynb)

## Introducción
### Justificación
### Casos de uso:
* Priorizar las interacciones letales sintéticas candidatas humanas basándose en pruebas previas de interacción en cribados de SL en levadura.
* Descubrimiento de novo de interacciones SL.
### Aproximación
Este cuaderno vuelve a implementar el enfoque descrito por Srivas et al. (2016).
### Utilización:
Añada los genes de interés a la variable "inputGenes" y ejecute el siguiente paso.
### Resumen del flujo de trabajo
### Conjuntos de datos
#### Interacciones letales sintéticas de la levadura
Constanzo et al. (2016)
#### Cartografía de ortólogos entre humanos y levadura
En el cuaderno adjunto (Mapping human to yeast orthologs) se explica cómo se han descargado y guardado en tablas de Bigquery los datos del mapeo de ortólogos entre humanos y levadura.
### References
* Costanzo M, VanderSluis B, Koch EN, Baryshnikova A, Pons C, Tan G, Wang W, Usaj M, Hanchard J, Lee SD, Pelechano V, Styles EB, Billmann M, van Leeuwen J, van Dyk N, Lin ZY, Kuzmin E, Nelson J, Piotrowski JS, Srikumar T, Bahr S, Chen Y, Deshpande R, Kurat CF, Li SC, Li Z, Usaj MM, Okada H, Pascoe N, San Luis BJ, Sharifpoor S, Shuteriqi E, Simpkins SW, Snider J, Suresh HG, Tan Y, Zhu H, Malod-Dognin N, Janjic V, Przulj N, Troyanskaya OG, Stagljar I, Xia T, Ohya Y, Gingras AC, Raught B, Boutros M, Steinmetz LM, Moore CL, Rosebrock AP, Caudy AA, Myers CL, Andrews B, Boone C. **A global genetic interaction network maps a wiring diagram of cellular function.** Science. 2016 Sep 23;353(6306). pii: aaf1420. PubMed PMID: 27708008; PubMed Central PMCID: PMC5661885.
* Srivas R, Shen JP, Yang CC, Sun SM, Li J, Gross AM, Jensen J, Licon K, Bojorquez-Gomez A, Klepper K, Huang J, Pekin D, Xu JL, Yeerna H, Sivaganesh V, Kollenstart L, van Attikum H, Aza-Blanc P, Sobol RW, Ideker T. **A Network of Conserved Synthetic Lethal Interactions for Exploration of Precision Cancer Therapy**. Mol Cell. 2016 Aug 4;63(3):514-25. doi:10.1016/j.molcel.2016.06.022.Epub 2016 Jul 21. PubMed PMID: 27453043; PubMed Central PMCID: PMC5209245. 

## Preámbulo
Esta sección describe cómo configurar el entorno de análisis, incluida la autenticación de la plataforma en la nube de Google y la importación de todas las bibliotecas de Python necesarias.

### Configuración del entorno de análisis

In [1]:
# Este bloque de código instala las dependencias necesarias. Por favor, ejecútelo solo una vez, la primera ocasión que ejecute este cuaderno.
# Por favor, no lo ejecute si está corriendo el cuaderno en MyBinder.

!pip3 install google-cloud-bigquery
!pip3 install matplotlib
!pip3 install plotly
!pip3 install scipy
!pip3 install statsmodels
!pip3 install ipywidgets

In [2]:
# Importación de los módulos
from google.cloud import bigquery
import sys
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import stats 
import numpy as np
import json
import statsmodels.stats.multitest as multi
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
import plotly
import plotly.express as px
import pyarrow


### Autentificación de Google
Ejecutar las celdas de BigQuery en este cuaderno requiere un proyecto de Google Cloud y las instrucciones para crear uno pueden encontrarse en la [documentación de Google](https://cloud.google.com/resource-manager/docs/creating-managing-projects#console). Además, la instancia debe estar autorizada para facturar las consultas del proyecto.
Para más información sobre cómo iniciarse en la nube, consulte ['Quick Start Guide to ISB-CGC'](https://isb-cancer-genomics-cloud.readthedocs.io/en/latest/sections/HowToGetStartedonISB-CGC.html). Asimismo, puede localizar métodos de autenticación alternativos en la [documentación de Google](https://googleapis.dev/python/google-api-core/latest/auth.html).

#### * Si está ejecutando este cuaderno en un entorno de máquina virtual de Google (Compute Engine, App Engine, Cloud Run, Cloud Functions): 
La autenticación debería "simplemente funcionar".

#### * Si la ejecución se realiza en un entorno local: 
La forma más sencilla de autenticarse es emplear el SDK de Google Cloud.

In [ ]:
#Por favor, no ejecute este bloque de código si está corriendo el cuaderno en MyBinder.
#Los usuarios deben ejecutar el siguiente comando en su máquina local o a través del cuaderno.
#Asegúrese de instalar la nube de Google en el entorno local. Para más detalles sobre la instalación de gcloud, consulte el soporte en https://cloud.google.com/sdk/docs/install

!gcloud auth application-default login

###

In [3]:
# Sustituya syntheticlethality por el ID de su proyecto
project_id='crucial-minutia-407312'
client = bigquery.Client(project_id) 

In [4]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [5]:
# Para utilizar su propia lista de genes, cópiela en la carpeta "Listas_genes" y reemplace el nombre del archivo_actual ("Genes_AmiGO2_ruta_biosintesis_NAD.txt") por el del suyo.
# Abrir el archivo de genes
archivo_actual = 'Listas_genes/Genes_sobreexpresados_UALCAN_HNSC.txt'
with open(archivo_actual, 'r') as file:
    # Leer las líneas del archivo y eliminar los espacios en blanco al principio y al final de cada línea
    genes = [line.strip() for line in file.readlines()]

# Unir los genes en una sola cadena utilizando comillas simples y separados por comas
inputGenes = ','.join([f"'{gene}'" for gene in genes])

In [6]:
# Opcionalmente puede ejecutarse este bloque de código, para verificar que la lista de genes se ha almacenado correctamente y que tiene el formato adecuado.
inputGenes
# Si el resultado es el esperado, "'GENE1', 'GENE2', ... 'GENEn'", entoces puede proseguir con el el siguiente paso.

"'CA9','MMP11','INHBA','MMP9','HOXD11','MMP12','ADAM12','LAMC2','HOXB7','ISG15','POSTN','AIM2','CDKN2A','MMP1','PTHLH','TGFBI','COL5A2','TM4SF19','COL5A1','COL1A1','NELL2','MMP3','FN1','IFI6','PCSK9','MFAP2','LOXL2','CXCL10','SERPINE1','MMP10','COL4A6','CTHRC1','LAMA3','COL12A1','CXCL13','ENO2','FST','C1QTNF6','GBP5','C7orf10','LEPREL1','COL3A1','COL4A1','SPP1','BST2','OASL','COL1A2','UCHL1','PLAU','FAP','IGF2BP2','RSAD2','CHST2','COL6A3','LY6K','NRG1','ANO1','NXPH4','COL7A1','ITGA5','CXCL9','AMIGO2','SULF1','ODZ2','SERPINH1','FSTL3','COL5A3','PMEPA1','IFIT1','MYBL2','COL4A2','COL4A5','DFNA5','RPL39L','IFIT3','COL27A1','COL6A1','IFI44','FEZ1','TNC','RBP1','PDPN','GREM1','IFI27','FOXM1','IDO1','NETO2','CHST11','PKMYT1','CDCA5','CDH11','PXDN','AKR1C1','EPSTI1','AKR1C3','NID1','BIRC5','TROAP','MCM2','SPARC','ITGA6','ADAM8','ALG1L','RFC4','CSPG4','CDC45','SOCS1','USP18','DCBLD1','LUM','PLEK2','CDC6','CENPA','KIF2C','GNLY','NPNT','ODC1','FADS3','AURKB','TK1','CTSL2','CEP55','UBE2C','FADD','

In [16]:
# Alternativamente, aunque solo se recomienda si su lista de genes es breve, puede introducir manualmente los genes cambiando el siguiente ejemplo: 
inputGenes = ["DDX3X","DICER1","DROSHA","TNFRSF14","TRAF7","TSC1",'POLG',
              "FBXO11","PRDM1","RFWD3","AMER1","LZTR1","ATP2B3"]
inputGenes = ["'"+x+"'" for x in inputGenes]
inputGenes = ','.join(inputGenes)
inputGenes

"'DDX3X','DICER1','DROSHA','TNFRSF14','TRAF7','TSC1','POLG','FBXO11','PRDM1','RFWD3','AMER1','LZTR1','ATP2B3'"

## Mapear ortólogos de levadura y obtener insteracciones SL
Para inferir las interacciones genéticas, se compara el crecimiento de las colonias de cepas doblemente mutantes con el de las cepas mutantes simples como medida de la aptitud del organismo debido a la esencialidad de los genes. Si la viabilidad celular de una colonia de mutantes dobles es mayor o menor que la de dos mutantes simples, se deducen interacciones genéticas positivas o negativas utilizando una métrica cuantitativa de fitness o puntuación genérica de interacción. Las interacciones letales sintéticas se definen como las interacciones genéticas con puntuaciones negativas bajas (< -0,35) en el extremo de la distribución de puntuaciones de interacción. Aprovechando este conjunto de datos, mapeamos genes de levadura con genes humanos utilizando información ortológica levadura-humano para identificar presuntos pares letales sintéticos humanos conservados. Los parámetros configurables son los siguientes:

Encontrar las parejas letales sintéticas de los genes de la lista proporcionada.

**Parámetros:**

**cutoff_algorithmMatchNo**: es el umbral mínimo de coincidencia deseado para que una comparación de genes levadura-humano se considere un ortólogo.

**cutoff_score**: el límite deseado de la métrica de aptitud cuantitativa. El valor predeterminado es (< 0,35), que corresponde a la cola izquierda de la distribución (<-0,35).

**cutoff_p**: el umbral de significación deseado. El valor por defecto es p < 0,05.


In [7]:
sql = '''
WITH
--- Retrieve YeastSymbols mapped to HumanSymbols for the input genes
INPUT_H2Y AS (
  SELECT YeastSymbol
    FROM `isb-cgc-bq.annotations_versioned.Human2Yeast_mapping_Alliance_for_Genome_Resources_R3_0_1`
    
   WHERE HumanSymbol IN (__INPUTGENES__) AND
         AlgorithmsMatch >= __ALGORITHMCUTOFF__
),

--- Identify protein-protein interactions using the YeastSymbols (left match)
Yeast_ITX1 AS (
  SELECT UPPER(Query_allele_name)       AS Interactor1, 
         UPPER(Array_allele_name)       AS Interactor2,
         Genetic_interaction_score_____ AS Interaction_score,
         P_value
    FROM `isb-cgc-bq.supplementary_tables.Constanzo_etal_Science_2016_SGA_Genetic_Interactions`
   WHERE (Genetic_interaction_score_____ < __SCORECUTOFF__ AND P_value < __PvalueCUTOFF__) AND
         (UPPER(Query_allele_name) IN (SELECT YeastSymbol FROM INPUT_H2Y))
   
),

--- Identify protein-protein interactions using the YeastSymbols (right match)
Yeast_ITX2 AS (
  SELECT UPPER(Array_allele_name)       AS Interactor1, 
         UPPER(Query_allele_name)       AS Interactor2,
         Genetic_interaction_score_____ AS Interaction_score,
         P_value
    FROM `isb-cgc-bq.supplementary_tables.Constanzo_etal_Science_2016_SGA_Genetic_Interactions`
   WHERE (Genetic_interaction_score_____ < __SCORECUTOFF__ AND P_value < __PvalueCUTOFF__) AND
         (UPPER(Array_allele_name) IN (SELECT YeastSymbol FROM INPUT_H2Y))
   
),

--- Union interaction tables
Union_ITX AS (
  SELECT * FROM Yeast_ITX1
   UNION ALL
  SELECT * FROM Yeast_ITX2
)

--- Convert YeastSymbols to HumanSymbols in the protein-protein interations
SELECT DISTINCT 
       GINFO1.EntrezID        AS EntrezID_Input,
       H2Y1.HumanSymbol       AS Gene_Input,
---       Add if you want to know what yeast genes are involved
---       YITX.Interactor1       AS Gene_Input_Yeast,
       GINFO2.EntrezID        AS EntrezID_SL_Candidate,
       H2Y2.HumanSymbol       AS Gene_SL_Candidate,
---       Add if you want to know what yeast genes are involved
---       YITX.Interactor2       AS Gene_SL_Candidate_Yeast,
       YITX.Interaction_score AS Interaction_score,
       YITX.P_value           AS P_value
       
  FROM Union_ITX AS YITX
       LEFT JOIN `isb-cgc-bq.annotations_versioned.Human2Yeast_mapping_Alliance_for_Genome_Resources_R3_0_1`                      AS H2Y1   ON YITX.Interactor1 = H2Y1.YeastSymbol
       LEFT JOIN `isb-cgc-bq.annotations_versioned.Human2Yeast_mapping_Alliance_for_Genome_Resources_R3_0_1`                      AS H2Y2   ON YITX.Interactor2 = H2Y2.YeastSymbol
       LEFT JOIN `isb-cgc-bq.synthetic_lethality.gene_info_human_HGNC_NCBI_2020_07` AS GINFO1 ON H2Y1.HumanID = GINFO1.HGNCID
       LEFT JOIN  `isb-cgc-bq.synthetic_lethality.gene_info_human_HGNC_NCBI_2020_07` AS GINFO2 ON H2Y2.HumanID = GINFO2.HGNCID
       
 WHERE (H2Y1.HumanSymbol IS NOT NULL AND YITX.Interactor1 IS NOT NULL) AND
       (H2Y2.HumanSymbol IS NOT NULL AND YITX.Interactor2 IS NOT NULL)

'''
# Seleccione los umbrales a utilizar
cutoff_algorithmMatchNo = "3"
cutoff_score = "-0.35"
cutoff_p = "0.01"

sql = sql.replace("__INPUTGENES__", inputGenes)
sql = sql.replace("__ALGORITHMCUTOFF__", cutoff_algorithmMatchNo)
sql = sql.replace("__SCORECUTOFF__", cutoff_score)
sql = sql.replace("__PvalueCUTOFF__", cutoff_p)

res = client.query(sql).to_dataframe()

## Get Yeast SL Interactions

In [8]:
# Lista de pares SL para los genes de entrada.
res

,EntrezID_Input,Gene_Input,EntrezID_SL_Candidate,Gene_SL_Candidate,Interaction_score,P_value
0,4751,NEK2,891,CCNB1,-0.5950,6.059000e-19
1,4751,NEK2,9133,CCNB2,-0.5950,6.059000e-19
2,63922,CHTF18,51371,POMP,-0.3510,1.728000e-47
3,891,CCNB1,292,SLC25A5,-0.4290,1.705000e-03
4,891,CCNB1,291,SLC25A4,-0.4290,1.705000e-03
...,...,...,...,...,...,...
615,6545,SLC7A4,5723,PSPH,-0.6843,0.000000e+00
616,6542,SLC7A2,5723,PSPH,-0.6843,0.000000e+00
617,6541,SLC7A1,5723,PSPH,-0.6843,0.000000e+00
618,57709,SLC7A14,5723,PSPH,-0.6843,0.000000e+00


**Gene_Input**: la lista de genes introducida por el usuario cuya(s) pareja(s) letal(es) sintética(s) se busca(n).

**EntrezID_Input**: muestra los identificadores Entrez de los genes de la lista de genes introducida por el usuario.

**EntrezID_SL_Candidate and Gene_SL_Candidate**: son los identificadores Entrez y los símbolos del HGNC para los pares letales sintéticos inferidos. 

**Interaction_score and P_value**: la estimación de la fuerza de interacción entre el gen de entrada y su pareja SL en la tabla isb-cgc-bq.supplementary_tables.Constanzo_etal_Science_2016_SGA_Genetic_Interactions.

Los resultados pueden guardarse en un archivo CSV y/o Excel.

In [10]:
#res.to_csv(path_or_buf='conserved_SL_output.csv', index=False)
res.to_excel(excel_writer='conserved_SL_output.xlsx', index=False)